In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-12233ac7-0827-4074-7e85-9da18093f928)


In [ ]:
!git clone https://gitlab.mff.cuni.cz/mazoureb/custom_fsdet.git

fatal: destination path 'custom_fsdet' already exists and is not an empty directory.


In [ ]:
!pip install torch==1.6.0 torchvision==0.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python3 -m pip install detectron2==0.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.6/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.6/index.html


In [ ]:
%cd custom_fsdet
!python3 -m pip install -r requirements.txt

/content/custom_fsdet
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
%cd custom_fsdet/

/content
Mounted at /content/gdrive
/content/custom_fsdet


In [ ]:
!ls /content/gdrive/MyDrive/fsdet-v

config.yaml
custom
events.out.tfevents.1658691737.38026711c178.1681.0
events.out.tfevents.1658732575.321faf86f6fe.663.0
events.out.tfevents.1658738132.2f167e10b4d7.677.0
events.out.tfevents.1659035890.f68983988098.632.0
events.out.tfevents.1659036248.f68983988098.723.0
jsons
last_checkpoint
log.txt
metrics.json
model_reset_surgery.pth


### Few shot training

In [ ]:
!nvidia-smi

Thu Jul 28 19:31:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python3 -m tools.train_net \
    --num-gpus 1 \
    --config-file configs/Custom-Bear/base_training_cfg.yaml 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

Command Line Args: Namespace(config_file='configs/Custom-Bear/base_training_cfg.yaml', dist_url='tcp://127.0.0.1:49152', end_iter=-1, eval_all=False, eval_during_train=False, eval_iter=-1, eval_only=False, machine_rank=0, num_gpus=1, num_machines=1, opts=None, resume=False, start_iter=-1)
[07/28 19:34:35 detectron2]: Rank of current process: 0. World size: 1
[07/28 19:34:35 detectron2]: Command line arguments: Namespace(config_file='configs/Custom-Bear/base_training_cfg.yaml', dist_url='tcp://127.0.0.1:49152', end_iter=-1, eval_all=False, eval_during_train=False, eval_iter=-1, eval_only=False, machine_rank=0, num_gpus=1, num_machines=1, opts=None, resume=False, start_iter=-1)

In [ ]:
assert False

AssertionError: ignored

In [ ]:
#!cp checkpoints/coco/faster_rcnn/faster_rcnn_R_101_FPN_ft_all_1shot/model_0002999.pth /content/gdrive/MyDrive/fsdet-v/

### Model testing

In [ ]:
!cp /content/gdrive/MyDrive/fsdet-v/jsons/datasplit/5k.json datasets/cocosplit/datasplit/

In [ ]:
# Download images of the validation set - 5k.json
%cd datasets/cocosplit/datasplit
!cp ../download_from_jsons.sh ./
!./download_from_jsons.sh
!wget -i links.txt -P ../../coco/val2014

In [ ]:
!python3 -m tools.test_net --num-gpus 1 \
  --config-file configs/COCO-detection/faster_rcnn_R_101_FPN_ft_all_1shot.yaml \
  --eval-only \
  --opts MODEL.WEIGHTS checkpoints/coco/faster_rcnn/faster_rcnn_R_101_FPN_ft_all_1shot/model_0002999.pth 